In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Carica il dataset
df = pd.read_csv('../dataset/preprocessed_dataset.csv')
df = df.drop('votes', axis=1)
df = df.drop('avg_vote', axis=1)

In [37]:
X = df.drop('revenue_with_CPI', axis=1)
y = df['revenue_with_CPI']
print(X.head())

   duration  converted_budget  dir_oscar_nomination  dir_oscar_won  \
0      88.0          175700.3                     0              0   
1      59.0         3013850.0                     0              0   
2      77.0          521727.6                     0              0   
3      50.0         5598468.6                     0              0   
4     300.0        10802441.1                     0              0   

   writer_oscar_nomination  writer_oscar_won  cast_oscar_nomination  \
0                        0                 0                      0   
1                        0                 0                      0   
2                        0                 0                      0   
3                        0                 0                      0   
4                        0                 0                      0   

   cast_oscar_won  dir_globe_nomination  dir_globe_won  ...  month_3  month_4  \
0               0                     0              0  ...    False   

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats
scaler = StandardScaler()
numerical_columns = ['duration','converted_budget',
                     'dir_oscar_nomination', 'dir_oscar_won', 
                     'writer_oscar_nomination', 'writer_oscar_won', 
                     'cast_oscar_nomination', 'cast_oscar_won', 
                     'dir_globe_nomination', 'dir_globe_won', 
                     'cast_globe_nomination', 'cast_globe_won',
                     'BAFTA_writer_nom','BAFTA_writer_won', 'dir_emmy_nom','dir_emmy_won','writer_emmy_won',
                     #'actors_films_before', 'director_films_before', 'writers_films_before'
                     ]

'''lambdas = {}
for col in numerical_columns:
    print(col)
    # La trasformazione di Box-Cox richiede valori positivi, aggiungi 1 se ci sono zeri o negativi
    X_train[col], lambdas[col] = stats.boxcox(X_train[col] + 1)
    X_test[col] = stats.boxcox(X_test[col]+1, lmbda=lambdas[col])'''
for col in numerical_columns:
     X_train[col] = np.log1p(X_train[col])
     X_test[col] = np.log1p(X_test[col])


scaler = StandardScaler()

# Adatta lo scaler solo al training set e poi trasforma sia il training set che il test set
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])


#X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)


# Esegui la PCA solo sul set di training
#pca = PCA(n_components=10)  # Sostituisci 10 con il numero desiderato di componenti principali
#X_train_resampled = pca.fit_transform(X_train_resampled)
#X_test = pca.transform(X_test)  # Applica le stesse trasformazioni al set di test

#y_train_transformed, lambda_train = stats.boxcox(y_train)
#y_test_transformed = stats.boxcox(y_test, lmbda=lambda_train)

#y_train_transformed = np.log1p(y_train)
#y_test_transformed = np.log1p(y_test)


In [40]:
from sklearn.ensemble import RandomForestRegressor

# Addestra un modello di Random Forest
rf = RandomForestRegressor(n_estimators=100,max_features=4, random_state=42)
rf.fit(X_train, y_train)

# Ottieni le importanze delle feature
importances = rf.feature_importances_

# Seleziona le feature con importanza sopra una soglia
threshold = np.percentile(importances, 75)  # Seleziona le feature nel top 25%
selected_features_rf = X_train.columns[importances >= threshold].tolist()

X_train = X_train[selected_features_rf]
X_test = X_test[selected_features_rf]
print("Selected features based on Random Forest importance:", selected_features_rf)


Selected features based on Random Forest importance: ['duration', 'converted_budget', 'writer_oscar_nomination', 'cast_oscar_nomination', 'cast_oscar_won', 'cast_globe_nomination', 'cast_globe_won', 'BAFTA_act_nom', 'act_emmy_nom', 'genre_Action', 'genre_Adventure', 'genre_Comedy', 'genre_Crime', 'genre_Drama', 'genre_Romance', 'genre_Thriller', 'language_English', 'production_company_Disney', 'production_company_Sony', 'month_1', 'month_4', 'month_6', 'month_9', 'month_10']


In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Addestra il modello Random Forest
rf_regressor = RandomForestRegressor(n_jobs=10, n_estimators=500, max_features=4)
rf_regressor.fit(X_train, y_train)

# Fai previsioni sui dati di test
y_pred_test = rf_regressor.predict(X_test)

# Fai previsioni sui dati di train
y_pred_train = rf_regressor.predict(X_train)

# Calcola le metriche di valutazione per il test set
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Calcola le metriche di valutazione per il train set
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)

print(f'Test Set - MSE: {mse_test}')
print(f'Test Set - R²: {r2_test}')
print(f'Test Set - MAE: {mae_test}')

print(f'\nTrain Set - MSE: {mse_train}')
print(f'Train Set - R²: {r2_train}')
print(f'Train Set - MAE: {mae_train}')


Test Set - MSE: 4.642763775112868e+16
Test Set - R²: 0.3759952918254178
Test Set - MAE: 85322637.91117847

Train Set - MSE: 7363007566664910.0
Train Set - R²: 0.9127623764168248
Train Set - MAE: 30686530.28275454


"from scipy.special import inv_boxcox\n\n# Applica la trasformazione inversa ai valori previsti\ny_pred_inverse = inv_boxcox(y_pred_test, lambda_train)\n# Applica la trasformazione inversa ai valori effettivi\ny_test_inverse = inv_boxcox(y_test_transformed, lambda_train)\n\n# Calcola l'errore assoluto o l'errore quadratico medio tra i valori previsti invertiti e i valori effettivi invertiti\nmae_inverse = mean_absolute_error(y_test_inverse, y_pred_inverse)\nmse_inverse = mean_squared_error(y_test_inverse, y_pred_inverse)\n\nprint(f'Mean Absolute Error (Inverse): {mae_inverse}')\nprint(f'Mean Squared Error (Inverse): {mse_inverse}')"